# File containing the process of creating the dataset for each of 5 Athens Stock Exchange stocks, applying Z-score normalization

In [1]:
# Importing libraries
import pandas as pd # type: ignore
import numpy as np # type: ignore
import matplotlib.pyplot as plt

In [2]:
# Load the data
data = pd.read_pickle('/Users/lamprosganias/Downloads/datedata/2024_02_03_04/1 - Data Mining/stocks.pkl')
data

,A,B,G,L,M,N,O
TPEIR,Publication Timestamp Subcategory ...,Publication Timestamp Subcategory ...,Publication Timestamp Subcategory ...,Publication Timestamp Subcategory ...,Publication Timestamp Subcategory ...,Publication Timestamp Subcategory ...,Publication Timestamp Subcategory ...
MYTIL,Publication Timestamp Subcategory ...,Publication Timestamp Subcategory ...,Publication Timestamp Subcategory ...,Publication Timestamp Subcategory ...,Publication Timestamp Subcategory ...,Publication Timestamp Subcategory ...,Publication Timestamp Subcategory ...
OPAP,Publication Timestamp Subcategory ...,Publication Timestamp Subcategory ...,Publication Timestamp Subcategory ...,Publication Timestamp Subcategory ...,Publication Timestamp Subcategory ...,Publication Timestamp Subcategory ...,Publication Timestamp Subcategory ...
PPC,Publication Timestamp Subcategory ...,Publication Timestamp Subcategory ...,Publication Timestamp Subcategory ...,Publication Timestamp Subcategory ...,Publication Timestamp Subcategory ...,Publication Timestamp Subcategory ...,Publication Timestamp Subcategory ...
HTO,Publication Timestamp Subcategory ...,Publication Timestamp Subcategory ...,Publication Timestamp Subcategory ...,Publication Timestamp Subcategory ...,Publication Timestamp Subcategory ...,Publication Timestamp Subcategory ...,Publication Timestamp Subcategory ...


In [3]:
# Keeping only LOB data of each stock as Dataframes
data_frames = {}
for index in data.index:
    for column in data:
        if column == "B":
            key = f'{index}_{column}'
            data_frames[key] = data.at[index, column]

def get_stock_data(index, column):
    key = f'{index}_{column}'
    if key in data_frames:
        return data_frames[key]
    else:
        return None

for index in data.index:
    for column in data.columns:
        if column == "B":
            if not data.at[index,column].empty:
                variable_name = f'{index}_{column}'
                locals()[variable_name] = get_stock_data(index, column)
                print(variable_name)

TPEIR_B
MYTIL_B
OPAP_B
PPC_B
HTO_B


In [4]:
def filter_largest_datasets(datasets_dict, top_n=10):
    """
    Filters the top N datasets with the largest length of data and returns their names.
    
    Parameters:
        datasets_dict (dict): A dictionary where keys are dataset names and values are datasets.
        top_n (int): Number of datasets to keep with the largest lengths. Default is 5.
    
    Returns:
        list: The names of the datasets with the largest lengths.
    """
    # Sort dataset names by the length of their corresponding datasets in descending order
    sorted_datasets = sorted(datasets_dict.items(), key=lambda item: len(item[1]), reverse=True)
    
    # Return the top N dataset names
    return [name for name, _ in sorted_datasets[:top_n]]

datasets_dict = {
    "TPEIR_B": TPEIR_B,
    "MYTIL_B": MYTIL_B,
    "OPAP_B": OPAP_B,
    "PPC_B": PPC_B,
    "HTO_B": HTO_B
}
largest_datasets_names = filter_largest_datasets(datasets_dict, top_n=10)

# Print the results
print("Largest datasets:")
for name in largest_datasets_names:
    print(f"{name} with length {len(datasets_dict[name])}")

Largest datasets:
TPEIR_B with length 995550
PPC_B with length 414375
MYTIL_B with length 398478
HTO_B with length 366083
OPAP_B with length 363376


## Cleaning procedure on each stock's dataset.

In [5]:
largest_datasets = [TPEIR_B, MYTIL_B, OPAP_B, PPC_B, HTO_B]

# Processing each dataset
for i, df in enumerate(largest_datasets):
    df = df.drop(columns=["Subcategory", "Symbol", "Quote Levels"])
    df = df.drop_duplicates()
    df = df.dropna()
    df = df.sort_values(by=['Publication Timestamp'])
    df = df.set_index('Publication Timestamp')
    
    # Update the list with the processed DataFrame
    largest_datasets[i] = df

In [6]:
# Keeping only the columns that are needed.

columns_to_keep = [
    'bid_price_1', 'bid_size_1',
    'ask_price_1', 'ask_size_1',  
    'bid_price_2', 'bid_size_2',  
    'ask_price_2', 'ask_size_2',  
    'bid_price_3', 'bid_size_3', 
    'ask_price_3', 'ask_size_3',  
    'bid_price_4', 'bid_size_4',  
    'ask_price_4', 'ask_size_4',  
    'bid_price_5', 'bid_size_5', 
    'ask_price_5', 'ask_size_5',  
    'bid_price_6', 'bid_size_6',  
    'ask_price_6', 'ask_size_6',  
    'bid_price_7', 'bid_size_7', 
    'ask_price_7', 'ask_size_7',  
    'bid_price_8', 'bid_size_8', 
    'ask_price_8', 'ask_size_8', 
    'bid_price_9', 'bid_size_9', 
    'ask_price_9', 'ask_size_9',  
    'bid_price_10', 'bid_size_10',  
    'ask_price_10', 'ask_size_10', 
]

for i, df in enumerate(largest_datasets):
    df = df[columns_to_keep]
    df = df.dropna()
    largest_datasets[i] = df


In [7]:
# Applying multiplication to the bid and ask prices to have integers.

for i, df in enumerate(largest_datasets):
    for j in range(1, 11):
            df[f'bid_price_{j}'] *= 1000
            df[f'ask_price_{j}'] *= 1000
    largest_datasets[i] = df

In [8]:
# Convert data types to integers.

for i, df in enumerate(largest_datasets):
    # Check if the DataFrame has more than 40 rows
    if df.shape[1] > 40:
        df.iloc[:40, :] = df.iloc[:40, :].astype(int)
    else:
        df.iloc[:, :] = df.astype(int)
    largest_datasets[i] = df

/var/folders/10/ylf9z1m51bq7ks0c7rh20vfw0000gn/T/ipykernel_4793/532957413.py:8: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:, :] = df.astype(int)


In [9]:
# Filtering the data between 10:30 and 17:00.

for i, df in enumerate(largest_datasets):
    df = df.between_time('10:30', '17:00')
    largest_datasets[i] = df

## We create event representations for every 10 timestamps.

In [10]:
def aggregate_data(df, events_per_aggregation=10):
    """
    Aggregate every fixed number of events into one representation.
    
    Parameters:
    - df: DataFrame containing the LOB data.
    - events_per_aggregation: Number of events to aggregate into one representation.
    
    Returns:
    - Aggregated DataFrame.
    """
    n = len(df)
    n_aggregations = n // events_per_aggregation
    aggregated_data = []
    new_indices = []
    
    for i in range(n_aggregations):
        start = i * events_per_aggregation
        end = start + events_per_aggregation
        chunk = df.iloc[start:end]
        
        aggregated_row = {}
        for col in df.columns:
            if 'price' in col:
                aggregated_row[col] = chunk[col].mean()
            elif 'size' in col:
                aggregated_row[col] = chunk[col].sum()
        
        aggregated_data.append(aggregated_row)
        
        new_indices.append(chunk.index[-1])
    
    aggregated_df = pd.DataFrame(aggregated_data, index=new_indices)
    
    return aggregated_df

largest_datasets = [aggregate_data(df) for df in largest_datasets]


In [11]:
def apply_ema(df, columns, span):
    """
    Applies Exponential Moving Average (EMA) to specified columns.

    Parameters:
    - df: DataFrame containing the data.
    - columns: List of column names to apply EMA.
    - span: Span for the EMA.

    Returns:
    - DataFrame with EMA applied to specified columns.
    """
    df_ema = df.copy()
    for col in columns:
        df_ema[col] = df_ema[col].ewm(span=span, adjust=False).mean()
    return df_ema

# List of columns to apply EMA
price_columns = [f'bid_price_{i}' for i in range(1, 11)] + [f'ask_price_{i}' for i in range(1, 11)]

span = 10 
for i, df in enumerate(largest_datasets):
    df = apply_ema(df, price_columns, span)
    largest_datasets[i] = df

# Z-score normalization (Standardization) of previous day


In [12]:
# Apply normalization and convert the first 40 rows to integers
def previous_day_zscore_normalize(df):
    df_normalized = df.copy()
    
    # Extracting the date part from the timestamp
    df['date'] = df.index.date
    
    # Group by date to get daily statistics
    daily_stats = df.groupby('date').agg(['mean', 'std'])
    
    # Flatten the multi-index columns
    daily_stats.columns = ['_'.join(col) for col in daily_stats.columns]
    
    # Shift the statistics by one day
    daily_stats_shifted = daily_stats.shift(1)
    
    # Fill the first row of shifted statistics with the second row's values
    daily_stats_shifted.iloc[0] = daily_stats_shifted.iloc[1]
    
    # Join the shifted statistics back to the original dataframe
    df = df.join(daily_stats_shifted, on='date', rsuffix='_shifted')
    
    for column in df.iloc[:,:40]:
        mean_col = column + '_mean'
        std_col = column + '_std'

        df_normalized[column] = (df[column] - df[mean_col]) / df[std_col]
        df_normalized[column] = df_normalized[column].round(5)
    # Keep only the first 40 columns
    df_normalized = df_normalized.iloc[:, :40]
    
    # Check for extremely large values and handle them
    df_normalized[df_normalized > 1e5] = 1e5
    df_normalized[df_normalized < -1e5] = -1e5
    
    return df_normalized

largest_datasets_norm = largest_datasets.copy()

for i, df in enumerate(largest_datasets_norm):
    # Normalize the data
    df = previous_day_zscore_normalize(df)
    
    # Replace non-finite values with zeros before converting to integers
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.fillna(0, inplace=True)
    
    largest_datasets_norm[i] = df


In [13]:
# Calculate the mid_price and round it to 5 decimal places
for i, df in enumerate(largest_datasets_norm):
    df['mid_price'] = ((df["bid_price_1"] + df["ask_price_1"]) / 2).round(5)
    largest_datasets_norm[i] = df

In [15]:
case1_datasets = largest_datasets_norm.copy()


In [16]:
# Define the horizons and threshold range to test
horizons = [1, 2, 3, 5, 10]
thresholds = np.linspace(0, 0.1, 1000)

# Function to calculate labels for case1 based on a given threshold and horizon
def calculate_labels_case1(df, horizon, threshold):
    labels = pd.DataFrame(index=df.index)
    if horizon == 1:
        future_mid_prices = df['mid_price'].shift(-horizon)
    else:
        future_mid_prices = df['mid_price'].rolling(window=horizon, min_periods=1).mean().shift(-horizon)
    current_mid_prices = df['mid_price']
    percentage_change = ((future_mid_prices - current_mid_prices) / abs(current_mid_prices))
    label_column = f'label_{horizon}'
    labels[label_column] = 2  # Default label 2
    labels.loc[percentage_change >= threshold, label_column] = 1
    labels.loc[percentage_change <= -threshold, label_column] = 3
    return labels

# Function to find the best threshold for a single dataset and horizon
def find_best_threshold(df, horizon, thresholds):
    best_threshold = None
    best_balance = float('inf')
    
    for threshold in thresholds:

        labels = calculate_labels_case1(df, horizon, threshold)
        label_counts = labels[f'label_{horizon}'].value_counts(normalize=True)
        balance = abs(label_counts.get(1, 0.0) - 1/3) + abs(label_counts.get(2, 0.0) - 1/3) + abs(label_counts.get(3, 0.0) - 1/3)
        
        if balance < best_balance:
            best_balance = balance
            best_threshold = threshold
    
    return best_threshold

# Find the best thresholds for each horizon and dataset in case1
case1_thresholds = {}
for i, df in enumerate(case1_datasets):
    case1_thresholds[i] = {}
    for horizon in horizons:
        best_threshold = find_best_threshold(df, horizon, thresholds)
        case1_thresholds[i][horizon] = best_threshold

# Apply the best thresholds to add label columns to each dataset in case1_datasets
for i, df in enumerate(case1_datasets):
    for horizon in horizons:
        labels = calculate_labels_case1(df, horizon, case1_thresholds[i][horizon])
        df = df.join(labels)
        case1_datasets[i] = df

# Optional: Print results
print("Best thresholds for each dataset in case1:")
for i, thresholds in case1_thresholds.items():
    print(f"Dataset {i+1}: {thresholds}")


Best thresholds for each dataset in case1:
Dataset 1: {1: 0.0026026026026026027, 2: 0.0039039039039039042, 3: 0.005105105105105106, 5: 0.007607607607607608, 10: 0.013313313313313315}
Dataset 2: {1: 0.005105105105105106, 2: 0.0075075075075075074, 3: 0.009909909909909911, 5: 0.014214214214214215, 10: 0.024224224224224225}
Dataset 3: {1: 0.007607607607607608, 2: 0.011111111111111112, 3: 0.014614614614614616, 5: 0.02132132132132132, 10: 0.035835835835835834}
Dataset 4: {1: 0.006006006006006006, 2: 0.008708708708708709, 3: 0.011311311311311311, 5: 0.016216216216216217, 10: 0.027427427427427428}
Dataset 5: {1: 0.006206206206206206, 2: 0.009009009009009009, 3: 0.011811811811811812, 5: 0.017217217217217216, 10: 0.02972972972972973}


In [ ]:
# Define the stocks and corresponding datasets
stocks = ['TPEIR', 'MYTIL', 'OPAP', 'PPC', 'HTO']

# Define the date range for training and testing
train_start_date = '2024-01-31'
train_end_date = '2024-04-01'
test_start_date = '2024-04-01'
test_end_date = '2024-04-29'

# Function to process each dataset
def process_dataset(dataset, stock, case):
    if dataset.shape[0] > 5:
        dataset.iloc[:-5, :] = dataset.iloc[:-5, :].round(5)
    
    # Ensure the 'Publication Timestamp' is a datetime index if not already
    dataset.index = pd.to_datetime(dataset.index)
    dataset = dataset.sort_index()
    
    # Split into training and testing datasets
    train_dataset = dataset.loc[train_start_date:train_end_date]
    train_dataset = train_dataset.drop(columns="mid_price")
    train_dataset = train_dataset.T
    
    test_dataset = dataset.loc[test_start_date:test_end_date]
    test_dataset = test_dataset.drop(columns="mid_price")
    test_dataset = test_dataset.T
    
    # Export the datasets to text files
    train_filename = f'train_{stock}_case{case}.txt'
    test_filename = f'test_{stock}_case{case}.txt'
    
    train_dataset.to_csv(train_filename, header=False, index=False, sep=' ')
    test_dataset.to_csv(test_filename, header=False, index=False, sep=' ')
    
    # Extract the last 5 columns which are the labels
    labels_dataset = dataset.iloc[:, -5:]
    
    # Compute value counts for each label column
    value_counts = {col: labels_dataset[col].value_counts() for col in labels_dataset.columns}
    df_value_counts = pd.DataFrame(value_counts).fillna(0).astype(int)
    
    # Combine the value counts into a single DataFrame for display
    labels_value_counts = pd.concat([df_value_counts], axis=1, keys=[f'Data {case} - {stock}'])
    
    return labels_value_counts

# Process and export for case1_datasets
for i, (stock, dataset) in enumerate(zip(stocks, case1_datasets)):
    labels_value_counts_case1 = process_dataset(dataset, stock, case=1)
    print(labels_value_counts_case1)
